### Notebook purpose:
Defining what it means for a timestamped window (start, end) to be labeled as "y=1" i.e. disagreement.

In [1]:
import seaborn as sns
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import numpy as np

# import common analysis functions we've defined
parent_directory = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(parent_directory)
from common.annotation_utils import tp, get_union, plot_annotations

In [2]:
file_to_read = open("../outputs/unioned_annotations_dict.pkl", "rb")
unioned_annotations_dict = pickle.load(file_to_read)

# Print first entry of dictionary, to get sense of format
example_disagreements = unioned_annotations_dict[list(unioned_annotations_dict.keys())[0]]
print(f"Disagreement windows: {example_disagreements}\n")
print(f"Type of each window: {type(example_disagreements[0])}")

Disagreement windows: [(2.37,4.12), (9.32,11.27), (27.67,29.22)]

Type of each window: <class 'common.annotation_utils.tp'>


In [3]:
example_disagreement = example_disagreements[0]
print(f"Example disagreement: {example_disagreement}")

Example disagreement: (2.37,4.12)


In [4]:
# try different windows
# possible types are:
# 1. window ends before annotation start
# 2. window starts before annotation start, ends at annotation start
# 3. window starts before annotation start, ends between annotation start and end
# 4. window starts before annotation start, ends at annotation end
# 5. window starts before annotation start, ends after annotation end
# 6. window starts at annotation start, ends between annotation start and end
# 7. window starts at annotation start, ends at annotation end
# 8. window starts at annotation start, ends after annotation end
# 9. window starts after annotation start, ends between annotation start and end
# 10. window starts after annotation start, ends at annotation end
# 11. window starts after annotation start, ends after annotation end
# 12. window starts at annotation end, ends after annotation end
# 13. window starts after annotation end

unit_tests = [tp(0, 2), 
              tp(0, example_disagreement.start), 
              tp(0, 3), 
              tp(0, example_disagreement.end), 
              tp(0, 5), 
              tp(example_disagreement.start, 3), 
              tp(example_disagreement.start, example_disagreement.end), 
              tp(example_disagreement.start, 5), 
              tp(3, 4), 
              tp(3, example_disagreement.end), 
              tp(3, 5), 
              tp(example_disagreement.end, 5), 
              tp(5,10)]

def get_overlap(tp1, tp2):
    if (tp1.start <= tp2.end) and (tp2.start <= tp1.end):
        latest_start = max(tp1.start, tp2.start)
        earliest_end = min(tp1.end, tp2.end)
        delta = (earliest_end - latest_start)
        return delta / (tp1.end - tp1.start)
    else:
        return 0

for i in unit_tests:
    print(f"Example window: {i}")
    print(f"Disagreement window: {example_disagreement}")
    overlap_proportion = get_overlap(i, example_disagreement)
    print(f"Overlap proportion: {overlap_proportion}")
    print("")

Example window: (0.00,2.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 0

Example window: (0.00,2.37)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.0

Example window: (0.00,3.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.20834459280000006

Example window: (0.00,4.12)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.42424242423507796

Example window: (0.00,5.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.34999502212

Example window: (2.37,3.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 1.0

Example window: (2.37,4.12)
Disagreement window: (2.37,4.12)
Overlap proportion: 1.0

Example window: (2.37,5.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.6666486065816027

Example window: (3.00,4.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 1.0

Example window: (3.00,4.12)
Disagreement window: (2.37,4.12)
Overlap proportion: 1.0

Example window: (3.00,5.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.56

In [5]:
# Note that order matters in get_overlap: it's relative to the first argument
print(get_overlap(tp(0, 3), tp(2, 4)))
print(get_overlap(tp(2, 4), tp(0, 3)))

0.3333333333333333
0.5


## Testing the implementation in tp() class:

In [10]:
for i in unit_tests:
    print(f"Example window: {i}")
    print(f"Disagreement window: {example_disagreement}")
    print(f"Overlap proportion: {i.overlap(example_disagreement)}")
    print(example_disagreement in i)
    print("")

Example window: (0.00,2.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 0
False

Example window: (0.00,2.37)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.0
False

Example window: (0.00,3.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.20834459280000006
True

Example window: (0.00,4.12)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.42424242423507796
True

Example window: (0.00,5.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.34999502212
True

Example window: (2.37,3.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 1.0
True

Example window: (2.37,4.12)
Disagreement window: (2.37,4.12)
Overlap proportion: 1.0
True

Example window: (2.37,5.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 0.6666486065816027
True

Example window: (3.00,4.00)
Disagreement window: (2.37,4.12)
Overlap proportion: 1.0
True

Example window: (3.00,4.12)
Disagreement window: (2.37,4.12)
Overlap proportion: 1.0
True

Example window: (3.00,5.00)
Disag